In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time
from scipy.optimize import linear_sum_assignment

In [31]:
driver_df=pd.read_csv('driver points.csv')
pickup_df=pd.read_csv('pickup points.csv')

In [32]:
driver_df.head()

,Bus,d_lat,d_lon,institute
0,KA53D5162,12.855323,77.672600,MAHE
1,KA53D0166,13.069991,77.603710,MAHE
2,KA53B6965,13.123265,77.592183,MAHE
3,KA53D8420,12.924835,77.649409,MAHE
4,KA53D4606,12.894084,77.533930,MAHE


In [33]:
pickup_df.head()

,Bus,p_lat,p_lon,institute
0,KA53D5162,12.907820,77.604260,MAHE
1,KA53D0166,12.955736,77.592776,MAHE
2,KA53B6965,13.049924,77.592684,MAHE
3,KA53D8420,12.917237,77.623038,MAHE
4,KA53D4606,12.937688,77.519786,MAHE


In [34]:
base_url= "http://localhost:5000/route/v1/driving"

In [35]:
def osrm_calculate(lat1, lon1, lat2, lon2): #input parameters as default lat,lon
    src=f'{lon1},{lat1};{lon2},{lat2}'
    curl=f"http://localhost:5000/route/v1/driving/{src}?overview=false"
    time.sleep(0.2)
    try:
        json_response=requests.get(curl).json()
        return json_response['routes'][0]['distance']/1000
    except Exception as exc:
        print(f"OSRM error at {lat1},{lon1} to {lat2},{lon2}: {exc}")
        return np.inf

In [36]:
bus_coords = driver_df.set_index('Bus')[['d_lat', 'd_lon']].to_dict('index')
buses = list(bus_coords.keys())

pickup_coords = pickup_df.set_index('Bus')[['p_lat', 'p_lon']].to_dict('index')
pickups = list(pickup_coords.keys())

distance_mat=pd.DataFrame('', index=buses, columns=pickups)
distance_mat[buses[0]][pickups[0]]

''

In [37]:
for bus in buses:
    # print(bus)
    for pickup in pickups:
        # print(f"{bus} : {bus_coords[bus]['d_lat']}, {bus_coords[bus]['d_lon']} to {pickup_coords[pickup]['p_lat']}, {pickup_coords[pickup]['p_lon']} : {pickup}")
        distance=osrm_calculate(bus_coords[bus]['d_lat'], bus_coords[bus]['d_lon'], pickup_coords[pickup]['p_lat'], pickup_coords[pickup]['p_lon'])
        # print(distance)
        distance_mat.loc[pickup, bus]=distance
        # distance=osrm_calculate(bus_coords[bus]['d_lat']
        # print(f"{bus_coords[bus]['d_lat']}, {bus_coords[bus]['d_lon']} to {pickup_coords[pickup]['p_lat']}, {pickup_coords[bus]['p_lon']} : {pickup}")
distance_mat.head()

,KA53D5162,KA53D0166,KA53B6965,KA53D8420,KA53D4606,KA53D8419,KA53D5165,KA53D0158,KA07A9086,KA53D2958,KA53D2719,KA53B6967,KA53B6964,KA07A9089,KA53AB1814
KA53D5162,13.0193,22.5424,30.2996,7.6679,10.7298,38.3181,18.2889,24.3804,21.8084,21.8343,3.9033,33.7033,30.2996,33.9641,28.9248
KA53D0166,15.6728,15.9934,23.7507,8.6639,15.2581,29.6953,13.5563,17.8314,16.3009,16.3268,6.589,27.1543,23.7507,27.4152,22.3759
KA53B6965,28.2532,5.0737,13.3715,21.2443,24.5027,20.3448,14.4141,8.3962,21.993,22.019,19.1695,15.6275,13.3715,15.8884,10.9684
KA53D8420,9.9109,21.9304,29.6876,4.3638,13.7566,35.6322,19.9199,23.7684,18.6713,18.6972,0.8272,33.0913,29.6876,33.3521,28.3128
KA53D4606,23.2265,23.1808,30.938,17.7401,7.6461,29.8477,9.4005,22.3724,26.782,26.8079,14.1427,34.3417,30.938,34.6025,24.7965


In [38]:
optim_drivers, optim_pickups=linear_sum_assignment(distance_mat)

In [39]:
optimized_df = pd.DataFrame({
    'From_Bus': distance_mat.index[optim_drivers],
    'To_Bus': distance_mat.columns[optim_pickups],
    'Distance_km': distance_mat.values[optim_drivers, optim_pickups]
})

In [40]:
optimized_df.to_excel('Optimized_phase3.xlsx', index=False)